In [ ]:
import cv2
import mediapipe as mp
import httplib2

http  = httplib2.Http(timeout = 0.3)
url_forward = "http://192.168.4.1/FORWARD"
url_right = "http://192.168.4.1/RIGHT"
url_left = "http://192.168.4.1/LEFT"
url_stop = "http://192.168.4.1/STOP"
url_back = "http://192.168.4.1/BACKWARD"

overlaylist = list()
wCam, hCam = 648, 480
pTime = 0  # previous time
cTime = 0  # current time
cap = cv2.VideoCapture(0)
#cap.set(1,pTime)
#cap.set(2,cTime)


# in this var finger counts will be stored
count = [0,0,0,0,0]
counts = 0
lmlist = []
mpHands = mp.solutions.hands
hands = mpHands.Hands()


    
#draw the detected key points for us so that we don’t have to draw them manually.
mpDraw = mp.solutions.drawing_utils       

if cap.isOpened() == False:
    print('Error while opening the camera')
while  True:
    ret , image = cap.read()
    image = cv2.resize(image,(700,500))
    image = cv2.flip(image, 1)
    
    #image[0:200,0:200] = overlaylist[0]
    imgRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)                 # find the hand in image 'imgRGB'
    #print(results.multi_hand_landmarks)
   
     # check if the hand was dectected or not by command 'results.multi_hand_landmarks'
    if results.multi_hand_landmarks:        # results.multi_hand_landmarks contains hands landmarks
        # maybe two landmarks of two hands are dtected, therefore  we need for loop
        for handLms in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(image,handLms,mpHands.HAND_CONNECTIONS) # draw landmarks
            lmlist = []
            for id, lm in enumerate(handLms.landmark):
                
                #print(id, lm)
                h, w, c = image.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmlist.append([id,cx,cy])
                
            # check which finger is opened
            if lmlist[8][2] < lmlist[6][2]:
                count[0] = 1
                
            if lmlist[12][2] < lmlist[10][2]:
                count[1] = 1   
            
            if lmlist[16][2] < lmlist[14][2]:
                count[2] = 1
                
            if lmlist[20][2] < lmlist[18][2]:
                count[3] = 1 
                
            if lmlist[4][1] < lmlist[2][1]:
                count[4] = 1        
            #print(id,cx,cy)
                #cv2.circle(image,(cx,cy),15,(255,0,255),cv2.FILLED)
            counts = count[0]+count[1]+count[2]+count[3]+count[4] 
            break

    else:
        counts = 0
    
    # accessing the NodeMCU server and sending appropriate request using GET Method 
    if counts != 0:
        if counts == 5:
            try:
                response, content = http.request(url_forward,"GET")
            except:
                print("Did not get response from NodeMCU")
        elif counts == 1:
            try:
                response, content = http.request(url_right,"GET")
            except:
                print("Did not get response from NodeMCU")  
                
        elif counts == 2:
            try:
                response, content = http.request(url_left,"GET")
            except:    
                print("Did not get response from NodeMCU")
        elif counts == 3:
            try:
                response, content = http.request(url_back,"GET")
            except: 
                print("Did not get response from NodeMCU")
       # cv2.putText(image,'Finger counts: '+str(counts), (10, 70), cv2.FONT_HERSHEY_PLAIN, 2,
        #            (255, 255, 0), 2)
        
    else:
       #cv2.putText(image,'Finger counts: ' ,(10, 70), cv2.FONT_HERSHEY_PLAIN, 2,
         #           (255, 255, 0), 2)
        try:    
            response, content = http.request(url_stop,"GET")   
        except:
            print("Did not get response from NodeMCU")
            
    count = [0,0,0,0,0]
    cv2.imshow("Image", image)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 

Did not get response from NodeMCU
Did not get response from NodeMCU
Did not get response from NodeMCU
Did not get response from NodeMCU
Did not get response from NodeMCU
Did not get response from NodeMCU
Did not get response from NodeMCU
Did not get response from NodeMCU
